# **IMPORT**

## Import the libraries

In [1]:
import gzip
import pandas as pd

## Load the datasets

"[...] ratings have been taken from www.imdb.com"

Download: https://datasets.imdbws.com/

Documentation: https://developer.imdb.com/non-commercial-datasets/


In [2]:
movie_basics_path = f'./Datasets/title.basics.tsv.gz'

# Open the gzipped file
with gzip.open(movie_basics_path, 'rt', encoding='utf-8') as f:
    # Specify '\t' as the separator for TSV files
    df_basics = pd.read_csv(f, sep='\t', encoding='utf-8', low_memory=False)

# Drop columns titleType, primaryTitle, originalTitle, isAdult, runtimeMinutes
df_basics = df_basics.drop(columns=['titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'runtimeMinutes'])

# Drop rows with any NaN value
df_basics.replace('\\N', pd.NA, inplace=True)
df_basics = df_basics.dropna()

# Drop column endYear
df_basics = df_basics.drop(columns=['endYear'])

df_basics.head()

,tconst,startYear,genres
35171,tt0035803,1940,"Documentary,News"
37599,tt0038276,1946,Talk-Show
38433,tt0039120,1947,"Family,Game-Show"
38434,tt0039121,1947,Family
38436,tt0039123,1947,Drama


In [3]:
file_path_movie_crew = f'./Datasets/title.crew.tsv.gz'

# Open the gzipped file and read it using pandas
with gzip.open(file_path_movie_crew, 'rt') as f:
    # Specify '\t' as the separator for TSV files
    df_crew = pd.read_csv(f, sep='\t', encoding='utf-8')

# Drop rows with any NaN value
df_crew.replace('\\N', pd.NA, inplace=True)
df_crew = df_crew.dropna()
    
df_crew.head() # Notice directors and authors are stored by their own IDs

,tconst,directors,writers
8,tt0000009,nm0085156,nm0085156
34,tt0000036,nm0005690,nm0410331
74,tt0000076,nm0005690,nm0410331
89,tt0000091,nm0617588,nm0617588
106,tt0000108,nm0005690,nm0410331


In [4]:
file_path_movie_ratings = f'./Datasets/title.ratings.tsv.gz'

# Open the gzipped file and read it using pandas
with gzip.open(file_path_movie_ratings, 'rt') as f:
    # Specify '\t' as the separator for TSV files
    df_ratings = pd.read_csv(f, sep='\t', encoding='utf-8')

# Drop rows with any NaN value
df_ratings.replace('\\N', pd.NA, inplace=True)
df_ratings = df_ratings.dropna()

df_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2025
1,tt0000002,5.7,272
2,tt0000003,6.5,1967
3,tt0000004,5.4,178
4,tt0000005,6.2,2729


# **MERGE**

## Merge the three datasets on _tconst_

"In proposed model we use a pre filter before applying K-means algorithm. The attributes used to calculate distance of each point from centroid are Genre, Author, Director, Year, Rating."

In [5]:
# When merging, it only includes rows with matching 'tconst' values in all three DataFrames.
df = pd.merge(df_ratings, pd.merge(df_basics, df_crew, on='tconst'), on='tconst')

# Drop rows with any NaN value
df.replace('\\N', pd.NA, inplace=True)
df = df.dropna()

print(f"Only {len(df)} movies are left!") 
df.head()


Only 39008 movies are left!


,tconst,averageRating,numVotes,startYear,genres,directors,writers
0,tt0039123,8.0,218,1947,Drama,"nm0390776,nm2427430,nm0334353,nm0242409,nm1802...","nm1055756,nm0590316,nm0913670,nm0044801,nm0442..."
1,tt0040021,6.9,88,1948,Drama,"nm0696296,nm0781751,nm0912998,nm0000989,nm5625137","nm0048758,nm1497206,nm0070948,nm0116693,nm0129..."
2,tt0040033,6.2,19,1948,"Family,Game-Show","nm0343431,nm10484169,nm0228208,nm1055550",nm1257510
3,tt0040036,7.4,49,1948,Drama,"nm0626972,nm0232401,nm15065603,nm0076150,nm024...","nm2436797,nm0168641,nm1198517,nm0701090,nm0090..."
4,tt0040041,7.6,160,1948,"Comedy,Family","nm0370749,nm0308417,nm0000926,nm0143653,nm0809967","nm0549359,nm0459708,nm0009640,nm0123611,nm0224..."


# **PRE-PROCESS WEIGHTS**

## Compute the weight for each attribute of each movie

See "Weightage and matching of attributes" in section 3.3

In [6]:
# For each column in (averageRating, startYear, genres, directors, writers), we will create a new column with the weight

# Weight for averageRating is 1 if <5, and equals the rating if >=5, and it is multiplied by 2 if number of votes is >1000 and <=10000 and by 3 if >10000
df['ratingWeight'] = df.apply(lambda row: 1 if row['averageRating'] < 5 else row['averageRating'], axis=1)
df['ratingWeight'] = df.apply(lambda row: row['ratingWeight']*2 if row['numVotes'] > 1000 and row['numVotes'] <= 10000 else row['ratingWeight'], axis=1)
df['ratingWeight'] = df.apply(lambda row: row['ratingWeight']*3 if row['numVotes'] > 10000 else row['ratingWeight'], axis=1)

# Weight for the others is just the frequency of the value divided by the total number of movies
df['startYearWeight'] = df['startYear'].value_counts() / len(df)

# And, considering that genres, directors and writers are stored as strings with the IDs separated by commas

# Add temporary columns to store the sets
df['genres_set'] = df['genres'].apply(lambda x: set(x.split(',')))
df['directors_set'] = df['directors'].apply(lambda x: set(x.split(',')))
df['writers_set'] = df['writers'].apply(lambda x: set(x.split(',')))

# frozenset is used to make the sets hashable
# normalize=True returns the relative frequencies   
df['genresWeight'] = df['genres_set'].apply(frozenset).map(df['genres_set'].apply(frozenset).value_counts(normalize=True))
df['directorsWeight'] = df['directors_set'].apply(frozenset).map(df['directors_set'].apply(frozenset).value_counts(normalize=True))
df['writersWeight'] = df['writers_set'].apply(frozenset).map(df['writers_set'].apply(frozenset).value_counts(normalize=True))

# Drop temporary columns
df = df.drop(columns=['genres_set', 'directors_set', 'writers_set'])


KeyError: 'authors'

# **STORE**

## Save the filtered dataset as CSV

In [ ]:
# Define the output file path
output_file_path = f'./Datasets/dataset.csv'

# index=False -> Do not save the row index values to the CSV file
df.to_csv(output_file_path, index=False)

